# EP 1.1: 크롤링한 직업데이터 전처리하기

**주요내용**
- 📊 **데이터 전처리**: `CompanyIndustry` 항목에 대한 분류
- 🧹 **결측치 처리**: 
- 📈 **데이터 시각화**: 
- 🤖 **자동화된 분석**: 

## `CompanyIndustry` 항목에 대한 분류 

In [3]:
# 이전에 전처리한 csv파일을 Dataframe에 로드합니다.

import pandas as pd

df = pd.read_csv("data/pre_processed_jobs_worknet.csv")
df.keys()

Index(['JobTitle', 'JobVolunteerExperience', 'RequiredExperienceMonths',
       'PreferredExperienceMonths', 'JobVolunteerEducation', 'WeeklyWorkdays',
       'Workdays', 'JobPayment', 'MinMonthlyPayment', 'MaxMonthlyPayment',
       'JobContractType', 'JobBenefits', 'CompanyName', 'CompanyIndustry',
       'CompanySize', 'CompanyLocation'],
      dtype='object')

In [ ]:
test_df = df[['JobTitle', 'CompanyIndustry']]

test_df

,JobTitle,CompanyIndustry
0,단순 포장반직원 모집합니다,사료 도매업
1,단순포장 및 락크작업,"도금, 착색 및 기타 표면처리강재 제조업"
2,육류 가공 식당납품,육류 가공식품 도매업
3,[이천시 가좌리] 신축빌라 공사현장 잡역부 모집,단독 주택 건설업
4,생산직 채용(당사 제품 생산) 외국인 우대,구조용 금속 판제품 및 공작물 제조업
...,...,...
995,용접사(외국인력 우대) 정규직 모집,반도체 제조용 기계 제조업
996,"토마토,메론 하우스 재배 작업자 구인",채소작물 재배업
997,작물재배 단순노무,작물재배 및 축산 복합농업
998,레디얼 작업 및 단순사상원 (보조원) 모집,그 외 기타 특수목적용 기계 제조업


In [5]:
company_industries = df['CompanyIndustry'].dropna().to_list()

len(company_industries)

896

In [9]:
import seaborn as sns
from matplotlib import rc
import matplotlib.pyplot as plt

%matplotlib inline
# 한글 폰트 적용
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name="BM-K/KoSimCSE-bert",
    normalize=True,
)
# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/Users/heewungsong/anaconda3/envs/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name BM-K/KoSimCSE-bert. Creating a new one with MEAN pooling.


In [54]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def cosine_distance(embed_a, embed_b):
    """Calculate the cosine distance between two embeddings."""
    return 1 - np.dot(embed_a, embed_b) / (np.linalg.norm(embed_a) * np.linalg.norm(embed_b))
    # return cosine_similarity(np.array(embed_a), np.array(embed_b))

def euclidean_distance(embed_a, embed_b):
    """Calculate the Euclidean distance between two embeddings."""
    embed_a_ = np.array(embed_a)
    embed_b_ = np.array(embed_b)
    return np.linalg.norm(embed_a_ - embed_b_)

In [66]:
a = company_industries[0]
b = company_industries[4]
# a = "겨울"
# b = "춥다"

embed_a = embed_model.get_text_embedding(a)
embed_b = embed_model.get_text_embedding(b)


distance = cosine_distance(embed_a, embed_b)
print(f"The cosine distance between '{a}' and '{b}' is: {distance:.4f}")

distance2 = euclidean_distance(embed_a, embed_b)
print(f"The Euclidean distance between '{a}' and '{b}' is: {distance2:.4f}")

The cosine distance between '사료 도매업' and '구조용 금속 판제품 및 공작물 제조업' is: 0.7381
The Euclidean distance between '사료 도매업' and '구조용 금속 판제품 및 공작물 제조업' is: 1.2150


In [69]:
# 텍스트를 벡터로 변환
company_industries_embeddings = embed_model.get_text_embedding_batch(company_industries)

In [70]:
from sklearn.cluster import KMeans

# KMeans 클러스터링
kmeans = KMeans(n_clusters=8, random_state=42)
labels = kmeans.fit_predict(company_industries_embeddings)

# 결과를 데이터프레임에 추가
# df['Cluster_8'] = labels

/Users/heewungsong/anaconda3/envs/test/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# 'Cluster_8' 열의 각 값별로 개수를 세어 반환합니다.
labels.value_counts() 



# 클러스터 별 주요 산업 분야를 데이터프레임으로 구성
all_clusters = []
for cluster in sorted(labels.unique()):
    # 클러스터 별 산업 빈도수 계산
    cluster_data = df[labels == cluster]['CompanyIndustry'].value_counts().reset_index()
    cluster_data.columns = ['CompanyIndustry', 'Count']
    cluster_data['Cluster'] = cluster
    all_clusters.append(cluster_data)

# 모든 클러스터 데이터를 하나의 데이터프레임으로 합치기
final_df = pd.concat(all_clusters).reset_index(drop=True)

# 결과를 'Count' 컬럼 기준으로 내림차순 정렬
final_df = final_df.sort_values(by='Count', ascending=False)
# 클러스터 별로 정렬
final_df = final_df.sort_values(by='Cluster', ascending=True)

final_df

In [30]:
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# 간단한 전처리: 쉼표, 공백 등을 기준으로 토크나이징
# tokens = [industry.split() for industry in df['CompanyIndustry']]

# Word2Vec 모델 훈련
# model = Word2Vec(company_industries, vector_size=100, window=1, min_count=1, workers=4)

# 각 산업명을 벡터로 변환
# vectors = [model.wv[token] for token in company_industries]

# t-SNE를 사용하여 시각화
# tsne = TSNE(n_components=2, random_state=0)
# vectors_tsne = tsne.fit_transform(vectors)
# plt.figure(figsize=(6, 6))
# for i, label in enumerate(company_industries):
#     x, y = vectors_tsne[i, :]
#     plt.scatter(x, y)
#     plt.annotate(label, (x, y), textcoords="offset points", xytext=(0,10), ha='center')
# plt.show()